# Farrel & Lewandowsky, Chapter 2

In [1]:
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import Span
from bokeh.layouts import row, gridplot
output_notebook()

Loading BokehJS ...

## Listing 2.1

In [2]:
nreps = 10_000
nsamples = 2000

drift = 0
sdrw = 0.3
criterion = 3

latencies = np.zeros(nreps)
responses = np.zeros(nreps)
evidence = np.zeros((nreps, nsamples + 1))

for i in range(nreps):
    evidence[i, :] = np.cumsum(np.hstack((0, np.random.normal(drift, sdrw, nsamples))))
    p = np.argmax(np.abs(evidence[i, :]) > criterion)
    # if there is no threshold crossing, p will be 0, and consequently also responses[i] and latencies[i]
    responses[i] = np.sign(evidence[i, p])
    latencies[i] = p

## Listing 2.2

In [3]:
tbpn = min(nreps, 5)
fig = figure(sizing_mode = 'stretch_both',
    y_range=(-criterion - 0.5, criterion + 0.5),
    x_axis_label='Decision Time', y_axis_label='Evidence')
x = np.arange(nsamples + 1)
for i in range(tbpn):
    p = int(latencies[i])
    fig.line(x=x[:p], y=evidence[i, :p])
fig.add_layout(Span(location=criterion, line_dash='dashed'))
fig.add_layout(Span(location=-criterion, line_dash='dashed'))
show(fig)

## Listing 2.3

In [4]:
toprt = latencies[responses > 0]
topprop = len(toprt) / nreps
toprtmean = np.mean(toprt)
counts, bin_edges = np.histogram(toprt, bins=20, range=(0, max(latencies)))
width = np.diff(bin_edges) if len(toprt) > 0 else np.nan
x = bin_edges[0 : -1] + width / 2
figtop = figure(title=f'Top responses ({topprop}) m={toprtmean:.1f}',
    x_axis_label='Decision Time', y_axis_label='Frequency')
figtop.vbar(x=x, width=width, top=counts)

botrt = latencies[responses < 0]
botprop = len(botrt) / nreps
botrtmean = np.mean(botrt) if len(botrt) > 0 else np.nan
counts, bin_edges = np.histogram(botrt, bins=20, range=(0, max(latencies)))
width = np.diff(bin_edges)
x = bin_edges[0 : -1] + width / 2
figbot = figure(title=f'Bottom responses ({botprop}) m={botrtmean:.1f}',
    x_axis_label='Decision Time', y_axis_label='Frequency',
    x_range=figtop.x_range, y_range=figtop.y_range)
figbot.vbar(x=x, width=width, top=counts)

show(gridplot([figtop, figbot], ncols=2, sizing_mode = 'stretch_both'))

## Listing 2.4

In [5]:
nreps = 10_000
nsamples = 2000

drift = 1
sdrw = 0.3
criterion = 3
t2tsd = [0, 0.025]

latencies = np.zeros(nreps)
responses = np.zeros(nreps)
evidence = np.zeros((nreps, nsamples + 1))

for i in range(nreps):
    sp = np.random.normal(0, t2tsd[0])
    dr = np.random.normal(drift, t2tsd[1])
    evidence[i, :] = np.cumsum(np.hstack((sp, np.random.normal(dr, sdrw, nsamples))))
    p = np.argmax(np.abs(evidence[i, :]) > criterion)
    # if there is no threshold crossing, p will be 0, and consequently also responses[i] and latencies[i]
    responses[i] = np.sign(evidence[i, p])
    latencies[i] = p